# DIMPLE2 Simulation

DIMPLE2 is the counter-based extension of the ordinary DIMPLE system which aims to solve three major problems with DIMPLE:

1. Exponential space requirment in coupling precision
2. Underutilization of quadratic space in number of spins
3. Suboptimal convergence from deterministic dynamics

To this end, we propose an architecture which extends the buffer-chain DIMPLE hardware with three corresponding improvements:

1. Replace coupling cell's buffer chains with counters (sync/async)
2. Allow for multiple wave fronts in the array that can collide
3. Equip each coupling cell with a LFSR and thresholding circuit to jitter

The following is an event driven simulation of the proposed architecture with which to benchmark any FPGA implementations.

The following advantages it provides are:

- Solves problems outlined in opening statement
- Compute-in-memory design, efficient FPGA memory usage
- Parallel evaluation allows for faithful V2 model

Disadvantages are:

- Quadratic space usage in the number of spins
- Unresearched Mean-Field Collision proposal
- Requires one clock cycle per unit of delay (O(2^M), M-bit precision, slow!)

In [ ]:
class CouplingCell:

    def __init__(self,halt_period, noise_strategy=None):

        self.counter = 0
        self.halt_period = halt_period
        self.is_counting = False
        self.transmit = False
        self.valence = False

        if noise_strategy is None:
            self.noise_strategy = lambda x : 0
        else:
            self.noise_strategy = noise_strategy

class DIMPLE2:

    def __init__(self,spins,couplings,default_halt):

        # Initialize cells and normalize couplings
        self.cell_list = []
        self.default_halt = default_halt
        couplings = [[default_halt + j for j in i] for i in couplings]

        # Store all CouplingCells in a 2D array
        for i in range(spins):
            self.cell_list.append([])
            for j in range(spins):
                self.cell_list[-1].append(CouplingCell(couplings[i][j]))
            self.cell_list[-1][-1].transmit = True

    def __len__(self):
        return len(self.cell_list[0])

    def update(self):

        # Find smallest amount of time
        counters = [cell.counter for c in self.cell_list for cell in c if cell.counter != 0]
        min_counter = min(counters)

        for i in range(len(self.cell_list)):
            for j in range(len(i)):

                # Get current cell, next cell in row and cell storing comparison valence for next cell
                current_cell, cuidx, cuidy = self.cell_list[i][j], i , j
                next_cell, neidx, neidy = self.cell_list[i][j+1%len(self)], i , j + 1 % len(self)
                compare, coidx, coidy = self.cell_list[j+1][(2*j+2-i)%len(self)], j, (2*j+2-i) % len(self)

                # Current cell propagates wave
                if current_cell.counter == min_counter:

                    current_cell.is_counting = False
                    current_cell.counter = 0
                    current_cell.valence != current_cell.valence

                    # No collision in transmission case
                    if not next_cell.is_counting:

                        next_cell.is_counting = True
                        if next_cell.valence != compare.valence:
                            next_cell.counter = next_cell.halt_period + next_cell.noise()
                        else:
                            next_cell.counter = self.default_halt + next_cell.noise()

                    # Collision in transmission case (assumed literal mean field)
                    else:
                        next_cell.is_counting = True
                        if next_cell.valence != compare.valence:
                            next_cell.counter += next_cell.halt_period + next_cell.noise()
                            next_cell.counter >>= 1
                        else:
                            next_cell.counter += self.default_halt + next_cell.noise()
                            next_cell.counter >>= 1

                # Current cell does not propagate wave
                elif current_cell.is_counting:
                    current_cell.counter -= min_counter

    model = DIMPLE2(2,[[1,1],[1,1]],10)
